# 데이터 전처리

## 0.환경준비

### 0.1 Import

In [1]:
# 기본 라이브러리 가져오기
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

### 0.2 Data Loading

In [2]:
# titanic
path = 'https://raw.githubusercontent.com/DA4BAM/dataset/master/titanic.0.csv'
titanic = pd.read_csv(path, usecols=['Survived','Pclass','Sex','Age','SibSp','Parch','Fare','Cabin','Embarked'])
titanic.head()

Survived  Pclass     Sex   Age  SibSp  Parch     Fare Cabin Embarked
0         0       3    male  22.0      1      0   7.2500   NaN        S
1         1       1  female  38.0      1      0  71.2833   C85        C
2         1       3  female  26.0      0      0   7.9250   NaN        S
3         1       1  female  35.0      1      0  53.1000  C123        S
4         0       3    male  35.0      0      0   8.0500   NaN        S

In [3]:
# airquality
path = 'https://raw.githubusercontent.com/DA4BAM/dataset/master/air2.csv'
air = pd.read_csv(path)
air.head()

Ozone  Solar.R  Wind  Temp        Date
0     41    190.0   7.4    67  1973-05-01
1     36    118.0   8.0    72  1973-05-02
2     12    149.0  12.6    74  1973-05-03
3     18    313.0  11.5    62  1973-05-04
4     19      NaN  14.3    56  1973-05-05

## 1.전처리

### 1) 데이터 분할

* x, y 분할하기

In [4]:
target = 'Survived'
x = titanic.drop(target, axis = 1)
y = titanic.loc[:, target]

* train, validation, test 분할은 머신러닝 과정에서 다룹니다.

### 2) NaN 조치

#### ① 결측치 확인

In [5]:
x.isna().sum()

Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Cabin       687
Embarked      2
dtype: int64

In [ ]:
plt.figure(figsize = (12,6))
sns.heatmap(x.isna())
plt.show()

#### ② 결측치 제거

In [ ]:
temp = x.copy()

In [ ]:
temp.shape

* 행 제거

In [ ]:
temp.dropna(axis=0, inplace = True)
temp.isna().sum()

In [ ]:
temp.shape

* 열 제거

In [6]:
# x에서 Cabin 열을 삭제합시다.
x = x.drop('Cabin', axis=1)

# 행,열의 수를 확인하시오.
x.shape

(891, 7)

#### ③ 결측치 채우기

* .fillna
    * 단일값 : .fillna(0)
    * 이전 값, 이후 값 : .fillna(method = 'ffill'),  method = 'bfill'

In [ ]:
air

In [ ]:
temp = air.copy()
temp.head(7)

* 단일값으로 채우기

In [ ]:
# 단일값으로 채우는 것처럼 조회
temp.fillna(0).head(7)

In [ ]:
temp.head(7)

In [ ]:
# 단일값으로 진짜 채우기
temp.fillna(0, inplace = True)
temp.head(7)

* 이전값, 이후값으로 채우기

In [ ]:
temp = air.copy()

temp.isna().sum()

In [ ]:
# 이전 값으로 채우기
temp.fillna(method = 'ffill').head(7)

In [ ]:
# 이후 값으로 채우기
# temp = air.copy()
temp.fillna(method='bfill').head(7)

* 전,후 사이값으로 채우기 
    * .interpolate(method = 'linear')

In [ ]:
temp.interpolate(method='linear').head(7)

* x 에서 값 채우기
    * Age에 대해 평균값으로, x['Age'].mean()
    * Embarked에 대해 최빈값으로 채워 봅시다. x['Embarked'].mode()

In [7]:
x.head()

Pclass     Sex   Age  SibSp  Parch     Fare Embarked
0       3    male  22.0      1      0   7.2500        S
1       1  female  38.0      1      0  71.2833        C
2       3  female  26.0      0      0   7.9250        S
3       1  female  35.0      1      0  53.1000        S
4       3    male  35.0      0      0   8.0500        S

In [8]:
x.isna().sum()

Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

In [9]:
# Age 컬럼에 대하여 평균값으로 채우기
x['Age'] = x['Age'].fillna( x['Age'].mean() )
x.isna().sum()

Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    2
dtype: int64

In [10]:
# Embarked 컬럼에 대하여 최빈값으로 채우기
x['Embarked'].mode()[0]

'S'

In [11]:
x['Embarked'].fillna(x['Embarked'].mode()[0], inplace=True)
x.isna().sum()

Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64

### 3) 가변수화

* 모델링을 위해서 모든 데이터 값은 숫자 여야 합니다.
* 숫자 형식이 아니라 의미상 숫자
    * 타이타닉 데이터의 pclass(객실등급)가 1,2,3 입니다. 이는 숫자 형식이지만 숫자의 의미는 아닙니다. (3등급 객실의 3이 1등급 객실의 1에 3배의 의미는 아니라는 말입니다.)
* 그러므로 모든 범주형 변수는 모두 (이미 0,1로 되어 있는 변수를 빼고) 가변수화를 수행해야 합니다.

In [12]:
x.head()

Pclass     Sex   Age  SibSp  Parch     Fare Embarked
0       3    male  22.0      1      0   7.2500        S
1       1  female  38.0      1      0  71.2833        C
2       3  female  26.0      0      0   7.9250        S
3       1  female  35.0      1      0  53.1000        S
4       3    male  35.0      0      0   8.0500        S

In [13]:
col_dumm = ['Pclass', 'Sex', 'Embarked']

x = pd.get_dummies(x, columns=col_dumm, drop_first=True)
x.head()

Age  SibSp  Parch     Fare  Pclass_2  Pclass_3  Sex_male  Embarked_Q  \
0  22.0      1      0   7.2500         0         1         1           0   
1  38.0      1      0  71.2833         0         0         0           0   
2  26.0      0      0   7.9250         0         1         0           0   
3  35.0      1      0  53.1000         0         0         0           0   
4  35.0      0      0   8.0500         0         1         1           0   

   Embarked_S  
0           1  
1           0  
2           1  
3           1  
4           1

* drop_first = True : 가변수화 시, 첫번째 범주 열은 삭제. ==> 하나를 빼도 모든 범주 표현 가능.

### 4) 스케일링

In [14]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

* 정규화(Normalization, MinMax 방식)
    * 모든 값을 0 ~ 1 로.

In [15]:
# 선언하기
scaler = MinMaxScaler()

# 만들고 적용 .fit_transform()   : .fit() + .transform()
x1 = scaler.fit_transform(x)
# 적용하고 나면 넘파이 어레이가 된다.

# 모델링할 때에는 넘파이 어레이여도 상관 없지만, 우리가 편하게 살펴보기 위해 데이터프레임으로 변환합니다.
x1 = pd.DataFrame(x1, columns = list(x))
x1.head()

Age  SibSp  Parch      Fare  Pclass_2  Pclass_3  Sex_male  Embarked_Q  \
0  0.271174  0.125    0.0  0.014151       0.0       1.0       1.0         0.0   
1  0.472229  0.125    0.0  0.139136       0.0       0.0       0.0         0.0   
2  0.321438  0.000    0.0  0.015469       0.0       1.0       0.0         0.0   
3  0.434531  0.125    0.0  0.103644       0.0       0.0       0.0         0.0   
4  0.434531  0.000    0.0  0.015713       0.0       1.0       1.0         0.0   

   Embarked_S  
0         1.0  
1         0.0  
2         1.0  
3         1.0  
4         1.0

* 표준화(Standardization)
    * 평균은 0, 표준편차 1 로

In [16]:
# 선언하기
scaler = StandardScaler()
ss_list = ['Age', 'SibSp', 'Parch', 'Fare']

In [18]:
x_ss = x[ss_list]
x_drop = x.drop(ss_list, axis=1)

In [21]:
# 만들고 적용 .fit_transform()   : .fit() + .transform()
x_ss = scaler.fit_transform(x_ss)
# 적용하고 나면 넘파이 어레이가 된다.

In [24]:
x_ss = pd.DataFrame(x_ss, columns=ss_list)
x_concat = pd.concat([x_ss, x_drop], axis=1)

In [25]:
# 모델링할 때에는 넘파이 어레이여도 상관 없지만, 우리가 편하게 살펴보기 위해 데이터프레임으로 변환합니다.
x2 = pd.DataFrame(x_concat)
x2.head()

Age     SibSp     Parch      Fare  Pclass_2  Pclass_3  Sex_male  \
0 -0.592481  0.432793 -0.473674 -0.502445         0         1         1   
1  0.638789  0.432793 -0.473674  0.786845         0         0         0   
2 -0.284663 -0.474545 -0.473674 -0.488854         0         1         0   
3  0.407926  0.432793 -0.473674  0.420730         0         0         0   
4  0.407926 -0.474545 -0.473674 -0.486337         0         1         1   

   Embarked_Q  Embarked_S  
0           0           1  
1           0           0  
2           0           1  
3           0           1  
4           0           1

* 값들을 살펴봅시다.

In [26]:
x.describe()

Age       SibSp       Parch        Fare    Pclass_2    Pclass_3  \
count  891.000000  891.000000  891.000000  891.000000  891.000000  891.000000   
mean    29.699118    0.523008    0.381594   32.204208    0.206510    0.551066   
std     13.002015    1.102743    0.806057   49.693429    0.405028    0.497665   
min      0.420000    0.000000    0.000000    0.000000    0.000000    0.000000   
25%     22.000000    0.000000    0.000000    7.910400    0.000000    0.000000   
50%     29.699118    0.000000    0.000000   14.454200    0.000000    1.000000   
75%     35.000000    1.000000    0.000000   31.000000    0.000000    1.000000   
max     80.000000    8.000000    6.000000  512.329200    1.000000    1.000000   

         Sex_male  Embarked_Q  Embarked_S  
count  891.000000  891.000000  891.000000  
mean     0.647587    0.086420    0.725028  
std      0.477990    0.281141    0.446751  
min      0.000000    0.000000    0.000000  
25%      0.000000    0.000000    0.000000  
50%      1.000000    0.000000    1.000000  
75%      1.000000    0.000000    1.000000  
max      1.000000    1.000000    1.000000

In [27]:
x1.describe()

Age       SibSp       Parch        Fare    Pclass_2    Pclass_3  \
count  891.000000  891.000000  891.000000  891.000000  891.000000  891.000000   
mean     0.367921    0.065376    0.063599    0.062858    0.206510    0.551066   
std      0.163383    0.137843    0.134343    0.096995    0.405028    0.497665   
min      0.000000    0.000000    0.000000    0.000000    0.000000    0.000000   
25%      0.271174    0.000000    0.000000    0.015440    0.000000    0.000000   
50%      0.367921    0.000000    0.000000    0.028213    0.000000    1.000000   
75%      0.434531    0.125000    0.000000    0.060508    0.000000    1.000000   
max      1.000000    1.000000    1.000000    1.000000    1.000000    1.000000   

         Sex_male  Embarked_Q  Embarked_S  
count  891.000000  891.000000  891.000000  
mean     0.647587    0.086420    0.725028  
std      0.477990    0.281141    0.446751  
min      0.000000    0.000000    0.000000  
25%      0.000000    0.000000    0.000000  
50%      1.000000    0.000000    1.000000  
75%      1.000000    0.000000    1.000000  
max      1.000000    1.000000    1.000000

In [28]:
x2.describe()

Age         SibSp         Parch          Fare    Pclass_2  \
count  8.910000e+02  8.910000e+02  8.910000e+02  8.910000e+02  891.000000   
mean   2.562796e-16  3.456519e-16  6.716164e-17 -4.373606e-17    0.206510   
std    1.000562e+00  1.000562e+00  1.000562e+00  1.000562e+00    0.405028   
min   -2.253155e+00 -4.745452e-01 -4.736736e-01 -6.484217e-01    0.000000   
25%   -5.924806e-01 -4.745452e-01 -4.736736e-01 -4.891482e-01    0.000000   
50%    0.000000e+00 -4.745452e-01 -4.736736e-01 -3.573909e-01    0.000000   
75%    4.079260e-01  4.327934e-01 -4.736736e-01 -2.424635e-02    0.000000   
max    3.870872e+00  6.784163e+00  6.974147e+00  9.667167e+00    1.000000   

         Pclass_3    Sex_male  Embarked_Q  Embarked_S  
count  891.000000  891.000000  891.000000  891.000000  
mean     0.551066    0.647587    0.086420    0.725028  
std      0.497665    0.477990    0.281141    0.446751  
min      0.000000    0.000000    0.000000    0.000000  
25%      0.000000    0.000000    0.000000    0.000000  
50%      1.000000    1.000000    0.000000    1.000000  
75%      1.000000    1.000000    0.000000    1.000000  
max      1.000000    1.000000    1.000000    1.000000

-----

# 카시트 판매량 데이터

![](https://cdn.images.express.co.uk/img/dynamic/24/590x/child-car-seat-986556.jpg?r=1532946857754)

* 비즈니스 상황
    * 고객사는 국내(미국)와 국외에서 카시트를 판매하는 회사입니다.
    * 최근 경쟁사의 공격적인 마케팅으로 매출이 감소하고 있습니다.
    * 데이터 분석을 위한 전처리를 수행해 봅시다.



|	변수명	|	설명	|	구분	|
|	----	|	----	|	----	|
|	Sales 	|	 각 지역 판매량(단위 : 1000개)	|	Target	|
|	CompPrice 	|	지역별 경쟁사 판매가격(달러)	|	feature	|
|	Advertising 	|	 각 지역, 회사의 광고 예산(단위 : 1000달러)	|	feature|
|	Income 	|	 지역 주민 평균소득(단위 : 1000달러)	|	feature	|
|	Population 	|	 지역 인구수(단위 : 1000명)	|	feature	|
|	Price 	|	 자사 지역별 판매가격(달러)	|	feature	|
|	ShelveLoc 	|	 진열상태	|	feature	|
|	Age 	|	 지역 인구의 평균 연령	|	feature	|
|	US 	|	 매장이 미국에 있는지 여부	|	feature	|
|	Urban 	|	 매장이 도시에 있는지 여부	|	feature	|
|	Education 	|	 평균학력수준(범주 : 11~17)	|	feature	|


## 0.환경준비

### 0.1 Import

In [29]:
# 기본 라이브러리 가져오기
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

### 0.2 Data Loading

In [30]:
path = 'https://raw.githubusercontent.com/DA4BAM/dataset/master/Carseat_train.csv'
data = pd.read_csv(path)
data.head()

Sales  CompPrice  Income  Advertising  Population  Price ShelveLoc  Age  \
0   3.72      139.0   111.0            5         310    132       Bad   62   
1   2.93      143.0    21.0            5          81    160    Medium   67   
2   4.53      114.0     NaN            0          97    125    Medium   29   
3   8.47      119.0    88.0           10         170    101    Medium   61   
4  11.85      136.0    81.0           15         425    120      Good   67   

   Education Urban   US  
0         13   Yes  Yes  
1         12    No  Yes  
2         12   Yes   No  
3         13   Yes  Yes  
4         10   Yes  Yes

## 1.데이터 탐색

* 기초 통계량
    * .describe()
    * 시리즈 혹은 데이터프레임에 붙여서 사용하는 메서드.
    * .T : transpose. 행/열 위치를 바꿈.

In [ ]:
data.info()

In [ ]:
data['ShelveLoc'].unique()

In [ ]:
data['Urban'].unique()

In [ ]:
data['US'].unique()

In [31]:
# data.describe().T
data.describe()

Sales   CompPrice      Income  Advertising  Population  \
count  300.000000  287.000000  288.000000   300.000000  300.000000   
mean     7.441867  125.344948   68.673611     6.600000  266.950000   
std      2.928141   14.551674   28.558597     6.569413  148.112339   
min      0.000000   86.000000   21.000000     0.000000   12.000000   
25%      5.357500  116.000000   42.000000     0.000000  136.250000   
50%      7.260000  125.000000   69.000000     5.000000  278.000000   
75%      9.352500  135.000000   93.000000    11.000000  400.000000   
max     16.270000  175.000000  120.000000    29.000000  509.000000   

            Price         Age   Education  
count  300.000000  300.000000  300.000000  
mean   115.930000   52.963333   13.906667  
std     22.149653   16.377098    2.601380  
min     49.000000   25.000000   10.000000  
25%    101.000000   39.000000   12.000000  
50%    117.000000   54.000000   14.000000  
75%    131.000000   65.000000   16.000000  
max    191.000000   80.000000   18.000000

* 진열상태 별 판매량 조회

In [32]:
data.groupby('ShelveLoc', as_index = False)['Sales'].mean()

ShelveLoc      Sales
0       Bad   5.407432
1      Good  10.439194
2    Medium   7.226707

## 2.데이터 전처리 

In [33]:
target = 'Sales'

### 2.1 데이터를 feature와 , target으로 분할

In [34]:
x = data.drop(target, axis = 1)
y = data.loc[:, target]

### 2.2 NaN에 대한 조치

In [35]:
x.isna().sum()

CompPrice      13
Income         12
Advertising     0
Population      0
Price           0
ShelveLoc       0
Age             0
Education       0
Urban           0
US              0
dtype: int64

In [36]:
from sklearn.impute import SimpleImputer

In [37]:
# 선언
imputer = SimpleImputer(strategy='mean')

# 대상 리스트
impute_list = ['CompPrice', 'Income']

# 피팅하고 적용하기
x[impute_list] = imputer.fit_transform(x[impute_list])
x.isna().sum()

CompPrice      0
Income         0
Advertising    0
Population     0
Price          0
ShelveLoc      0
Age            0
Education      0
Urban          0
US             0
dtype: int64

### 2.3 가변수화
* 범주형 데이터이면서 값이 0,1 로 되어 있는 것이 아니라면, 가변수화를 수행해야 합니다.
* 대상이 되는 변수에 대해서 가변수화를 수행해주세요.

In [40]:
x['Education'].unique()

array([13, 12, 10, 16, 11, 18, 17, 14, 15], dtype=int64)

In [41]:
dum_cols = ['ShelveLoc','US','Urban','Education']

x = pd.get_dummies(x, columns = dum_cols ,drop_first = True)
x.head()


CompPrice      Income  Advertising  Population  Price  Age  ShelveLoc_Good  \
0      139.0  111.000000            5         310    132   62               0   
1      143.0   21.000000            5          81    160   67               0   
2      114.0   68.673611            0          97    125   29               0   
3      119.0   88.000000           10         170    101   61               0   
4      136.0   81.000000           15         425    120   67               1   

   ShelveLoc_Medium  US_Yes  Urban_Yes  Education_11  Education_12  \
0                 0       1          1             0             0   
1                 1       1          0             0             1   
2                 1       0          1             0             1   
3                 1       1          1             0             0   
4                 0       1          1             0             0   

   Education_13  Education_14  Education_15  Education_16  Education_17  \
0             1             0             0             0             0   
1             0             0             0             0             0   
2             0             0             0             0             0   
3             1             0             0             0             0   
4             0             0             0             0             0   

   Education_18  
0             0  
1             0  
2             0  
3             0  
4             0

### 2.4 스케일링 
* 스케일링을 필요로 하는 알고리즘이 있습니다. 
* 필요하다고 판단될때 수행합니다.

* MinMaxScaler

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
col_x = list(x)
col_x

In [ ]:
scaler = MinMaxScaler()
x_s = scaler.fit_transform(x)

# 데이터프레임으로 다시 만듭시다.
x_s = pd.DataFrame(x_s, columns=col_x)
x_s.head()

In [ ]:
x.describe()

In [ ]:
x_s.describe()

* StandardScaler

In [38]:
from sklearn.preprocessing import StandardScaler

In [46]:
col_x = ['CompPrice','Income','Advertising','Population','Price','Age']
x_ss = x[col_x]
x_ss.head(1)

CompPrice  Income  Advertising  Population  Price  Age
0      139.0   111.0            5         310    132   62

In [47]:
x_drop = x.drop(col_x, axis=1)

In [49]:
x_drop.head(1)

ShelveLoc_Good  ShelveLoc_Medium  US_Yes  Urban_Yes  Education_11  \
0               0                 0       1          1             0   

   Education_12  Education_13  Education_14  Education_15  Education_16  \
0             0             1             0             0             0   

   Education_17  Education_18  
0             0             0

In [50]:
scaler = StandardScaler()
x_s = scaler.fit_transform(x_ss)

In [51]:
# 데이터프레임으로 다시 만듭시다.
x_s = pd.DataFrame(x_s, columns=col_x)
x_s.head()

CompPrice    Income  Advertising  Population     Price       Age
0   0.961077  1.515284    -0.243960    0.291143  0.726731  0.552709
1   1.242607 -1.706714    -0.243960   -1.257564  1.992972  0.858523
2  -0.798486  0.000000    -1.006335   -1.149357  0.410171 -1.465667
3  -0.446573  0.691884     0.518415   -0.655664 -0.675177  0.491546
4   0.749929  0.441285     1.280789    1.068878  0.184057  0.858523

In [54]:
x_s = pd.concat([x_s, x_drop], axis=1)

In [55]:
x.describe()

CompPrice      Income  Advertising  Population       Price  \
count  300.000000  300.000000   300.000000  300.000000  300.000000   
mean   125.344948   68.673611     6.600000  266.950000  115.930000   
std     14.231818   27.979646     6.569413  148.112339   22.149653   
min     86.000000   21.000000     0.000000   12.000000   49.000000   
25%    116.750000   42.000000     0.000000  136.250000  101.000000   
50%    125.344948   68.673611     5.000000  278.000000  117.000000   
75%    134.000000   92.000000    11.000000  400.000000  131.000000   
max    175.000000  120.000000    29.000000  509.000000  191.000000   

              Age  ShelveLoc_Good  ShelveLoc_Medium      US_Yes   Urban_Yes  \
count  300.000000      300.000000        300.000000  300.000000  300.000000   
mean    52.963333        0.206667          0.546667    0.650000    0.696667   
std     16.377098        0.405591          0.498649    0.477767    0.460466   
min     25.000000        0.000000          0.000000    0.000000    0.000000   
25%     39.000000        0.000000          0.000000    0.000000    0.000000   
50%     54.000000        0.000000          1.000000    1.000000    1.000000   
75%     65.000000        0.000000          1.000000    1.000000    1.000000   
max     80.000000        1.000000          1.000000    1.000000    1.000000   

       Education_11  Education_12  Education_13  Education_14  Education_15  \
count    300.000000    300.000000    300.000000    300.000000    300.000000   
mean       0.120000      0.143333      0.093333      0.110000      0.080000   
std        0.325504      0.350998      0.291385      0.313413      0.271746   
min        0.000000      0.000000      0.000000      0.000000      0.000000   
25%        0.000000      0.000000      0.000000      0.000000      0.000000   
50%        0.000000      0.000000      0.000000      0.000000      0.000000   
75%        0.000000      0.000000      0.000000      0.000000      0.000000   
max        1.000000      1.000000      1.000000      1.000000      1.000000   

       Education_16  Education_17  Education_18  
count    300.000000    300.000000    300.000000  
mean       0.120000      0.126667      0.096667  
std        0.325504      0.333155      0.295997  
min        0.000000      0.000000      0.000000  
25%        0.000000      0.000000      0.000000  
50%        0.000000      0.000000      0.000000  
75%        0.000000      0.000000      0.000000  
max        1.000000      1.000000      1.000000

In [56]:
x_s.describe()

CompPrice        Income   Advertising    Population         Price  \
count  3.000000e+02  3.000000e+02  3.000000e+02  3.000000e+02  3.000000e+02   
mean   9.747758e-16 -8.798517e-17  8.067621e-17  6.254256e-17 -3.160435e-16   
std    1.001671e+00  1.001671e+00  1.001671e+00  1.001671e+00  1.001671e+00   
min   -2.769196e+00 -1.706714e+00 -1.006335e+00 -1.724205e+00 -3.026767e+00   
25%   -6.049338e-01 -9.549148e-01 -1.006335e+00 -8.839127e-01 -6.751774e-01   
50%    1.000195e-15  0.000000e+00 -2.439599e-01  7.473019e-02  4.838847e-02   
75%    6.091642e-01  8.350843e-01  6.708897e-01  8.998056e-01  6.815086e-01   
max    3.494846e+00  1.837484e+00  3.415438e+00  1.636963e+00  3.394881e+00   

                Age  ShelveLoc_Good  ShelveLoc_Medium      US_Yes   Urban_Yes  \
count  3.000000e+02      300.000000        300.000000  300.000000  300.000000   
mean   1.465494e-16        0.206667          0.546667    0.650000    0.696667   
std    1.001671e+00        0.405591          0.498649    0.477767    0.460466   
min   -1.710319e+00        0.000000          0.000000    0.000000    0.000000   
25%   -8.540380e-01        0.000000          0.000000    0.000000    0.000000   
50%    6.340554e-02        0.000000          1.000000    1.000000    1.000000   
75%    7.361974e-01        0.000000          1.000000    1.000000    1.000000   
max    1.653641e+00        1.000000          1.000000    1.000000    1.000000   

       Education_11  Education_12  Education_13  Education_14  Education_15  \
count    300.000000    300.000000    300.000000    300.000000    300.000000   
mean       0.120000      0.143333      0.093333      0.110000      0.080000   
std        0.325504      0.350998      0.291385      0.313413      0.271746   
min        0.000000      0.000000      0.000000      0.000000      0.000000   
25%        0.000000      0.000000      0.000000      0.000000      0.000000   
50%        0.000000      0.000000      0.000000      0.000000      0.000000   
75%        0.000000      0.000000      0.000000      0.000000      0.000000   
max        1.000000      1.000000      1.000000      1.000000      1.000000   

       Education_16  Education_17  Education_18  
count    300.000000    300.000000    300.000000  
mean       0.120000      0.126667      0.096667  
std        0.325504      0.333155      0.295997  
min        0.000000      0.000000      0.000000  
25%        0.000000      0.000000      0.000000  
50%        0.000000      0.000000      0.000000  
75%        0.000000      0.000000      0.000000  
max        1.000000      1.000000      1.000000

-----

# 이동통신 고객 이탈 예측

* 여러분은 OO 통신회사 데이터분석가 입니다.
* 회사는 약정기간이 끝난 고객이 번호이동(이탈)해 가는 문제를 해결하고자 합니다.
* 어떤 고객이 번호이동(이탈)해 가는지 데이터분석을 의뢰하였습니다.
* 분석을 위한 데이터 전처리를 수행해 봅시다.

![](https://d18lkz4dllo6v2.cloudfront.net/cumulus_uploads/entry/23964/mobile%20phones.png)

## 0.환경설정

### 1) 라이브러리 로딩

In [57]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### 2) 데이터 로딩

In [58]:
# mobile data
data_path = "https://raw.githubusercontent.com/DA4BAM/dataset/master/mobile_NA2.csv"
data = pd.read_csv(data_path)
data.head()

id  COLLEGE  INCOME  OVERAGE  LEFTOVER     HOUSE  HANDSET_PRICE  \
0    544        1   47711      183        17  730589.0            192   
1    689        0   74132      191        43  535092.0            349   
2   3134        1  150419        0        14  204004.0            682   
3  12523        0  159567        0        58  281969.0            634   
4   2374        1   23392        0         0  216707.0            233   

   OVER_15MINS_CALLS_PER_MONTH  AVERAGE_CALL_DURATION REPORTED_SATISFACTION  \
0                           19                      5                 unsat   
1                           15                      2                 unsat   
2                            0                      6                 unsat   
3                            1                      1            very_unsat   
4                            0                     15                 unsat   

  REPORTED_USAGE_LEVEL CONSIDERING_CHANGE_OF_PLAN  CHURN  
0               little                considering      0  
1          very_little                         no      1  
2            very_high                considering      0  
3            very_high              never_thought      0  
4          very_little                         no      1

|변수 명|내용|구분|
|	----	|	----	|	----	|
|	COLLEGE	|	대학졸업 여부(1,0) - 범주	|		|
|	INCOME	|	연 수입액(달러)	|		|
|	OVERAGE	|	월 초과사용 시간(분)	|		|
|	LEFTOVER	|	월 사용 잔여시간비율(%)	|		|
|	HOUSE	|	집 가격(달러)	|		|
|	HANDSET_PRICE	|	핸드폰 가격(달러)	|		|
|	OVER_15MINS_CALLS_PER_MONTH	|	 평균 장기통화(15분 이상) 횟수	|		|
|	AVERAGE_CALL_DURATION	|	평균 통화시간(분)	|		|
|	REPORTED_SATISFACTION	|	만족도 설문('very_unsat', 'unsat', 'avg', 'sat', 'very_sat' ) - 범주	|		|
|	REPORTED_USAGE_LEVEL	|	사용 수준 설문('very_little', 'little', 'avg', 'high', 'very_high') - 범주	|		|
|	CONSIDERING_CHANGE_OF_PLAN	|	변경 계획 설문('never_thought', 'no', 'perhaps', 'considering',   'actively_looking_into_it') - 범주	|		|
|	**CHURN**	|	이탈여부(1 : 이탈, 0 : 잔류)	|	**Target**	|


----

## 1.데이터 탐색

## 2.데이터 전처리 

### 2.1 데이터 feature와 , target으로 분할

In [60]:
x = data.drop(['id', 'CHURN'], axis=1)
x.head(1)

COLLEGE  INCOME  OVERAGE  LEFTOVER     HOUSE  HANDSET_PRICE  \
0        1   47711      183        17  730589.0            192   

   OVER_15MINS_CALLS_PER_MONTH  AVERAGE_CALL_DURATION REPORTED_SATISFACTION  \
0                           19                      5                 unsat   

  REPORTED_USAGE_LEVEL CONSIDERING_CHANGE_OF_PLAN  
0               little                considering

In [61]:
y = data['CHURN']
y.head(1)

0    0
Name: CHURN, dtype: int64

### 2.2 NaN에 대한 조치
* 데이터에 포함된 NaN에 대해서 조치 방법을 결정하고 적용합니다.
* **조치방법을 결정한 이유**를 주석으로 기술합니다.

In [62]:
x.isna().sum()

COLLEGE                          0
INCOME                           0
OVERAGE                          0
LEFTOVER                         0
HOUSE                          215
HANDSET_PRICE                    0
OVER_15MINS_CALLS_PER_MONTH      0
AVERAGE_CALL_DURATION            0
REPORTED_SATISFACTION           38
REPORTED_USAGE_LEVEL             0
CONSIDERING_CHANGE_OF_PLAN       0
dtype: int64

In [63]:
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18000 entries, 0 to 17999
Data columns (total 11 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   COLLEGE                      18000 non-null  int64  
 1   INCOME                       18000 non-null  int64  
 2   OVERAGE                      18000 non-null  int64  
 3   LEFTOVER                     18000 non-null  int64  
 4   HOUSE                        17785 non-null  float64
 5   HANDSET_PRICE                18000 non-null  int64  
 6   OVER_15MINS_CALLS_PER_MONTH  18000 non-null  int64  
 7   AVERAGE_CALL_DURATION        18000 non-null  int64  
 8   REPORTED_SATISFACTION        17962 non-null  object 
 9   REPORTED_USAGE_LEVEL         18000 non-null  object 
 10  CONSIDERING_CHANGE_OF_PLAN   18000 non-null  object 
dtypes: float64(1), int64(7), object(3)
memory usage: 1.5+ MB


In [64]:
x['HOUSE'].mean()

492950.5326960922

In [65]:
x['HOUSE'] = x['HOUSE'].fillna(x['HOUSE'].mean())

In [66]:
x.isna().sum()

COLLEGE                         0
INCOME                          0
OVERAGE                         0
LEFTOVER                        0
HOUSE                           0
HANDSET_PRICE                   0
OVER_15MINS_CALLS_PER_MONTH     0
AVERAGE_CALL_DURATION           0
REPORTED_SATISFACTION          38
REPORTED_USAGE_LEVEL            0
CONSIDERING_CHANGE_OF_PLAN      0
dtype: int64

In [73]:
x_mode = x['REPORTED_SATISFACTION'].mode()[0]

In [74]:
x['REPORTED_SATISFACTION'] = x['REPORTED_SATISFACTION'].fillna(x_mode)

In [75]:
x.isna().sum()

COLLEGE                        0
INCOME                         0
OVERAGE                        0
LEFTOVER                       0
HOUSE                          0
HANDSET_PRICE                  0
OVER_15MINS_CALLS_PER_MONTH    0
AVERAGE_CALL_DURATION          0
REPORTED_SATISFACTION          0
REPORTED_USAGE_LEVEL           0
CONSIDERING_CHANGE_OF_PLAN     0
dtype: int64

### 2.3 가변수화
* 범주형 데이터이면서 값이 0,1 로 되어 있는 것이 아니라면, 가변수화를 수행해야 합니다.
* 대상이 되는 변수에 대해서 가변수화를 수행해주세요.

In [78]:
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18000 entries, 0 to 17999
Data columns (total 11 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   COLLEGE                      18000 non-null  int64  
 1   INCOME                       18000 non-null  int64  
 2   OVERAGE                      18000 non-null  int64  
 3   LEFTOVER                     18000 non-null  int64  
 4   HOUSE                        18000 non-null  float64
 5   HANDSET_PRICE                18000 non-null  int64  
 6   OVER_15MINS_CALLS_PER_MONTH  18000 non-null  int64  
 7   AVERAGE_CALL_DURATION        18000 non-null  int64  
 8   REPORTED_SATISFACTION        18000 non-null  object 
 9   REPORTED_USAGE_LEVEL         18000 non-null  object 
 10  CONSIDERING_CHANGE_OF_PLAN   18000 non-null  object 
dtypes: float64(1), int64(7), object(3)
memory usage: 1.5+ MB


In [77]:
x['COLLEGE'].head(10)

0    1
1    0
2    1
3    0
4    1
5    1
6    1
7    1
8    1
9    1
Name: COLLEGE, dtype: int64

In [79]:
col_list = ['COLLEGE', 'REPORTED_SATISFACTION', 'REPORTED_USAGE_LEVEL', 'CONSIDERING_CHANGE_OF_PLAN']

x = pd.get_dummies(x, columns=col_list, drop_first=True)

In [81]:
x.head()

INCOME  OVERAGE  LEFTOVER     HOUSE  HANDSET_PRICE  \
0   47711      183        17  730589.0            192   
1   74132      191        43  535092.0            349   
2  150419        0        14  204004.0            682   
3  159567        0        58  281969.0            634   
4   23392        0         0  216707.0            233   

   OVER_15MINS_CALLS_PER_MONTH  AVERAGE_CALL_DURATION  COLLEGE_1  \
0                           19                      5          1   
1                           15                      2          0   
2                            0                      6          1   
3                            1                      1          0   
4                            0                     15          1   

   REPORTED_SATISFACTION_sat  REPORTED_SATISFACTION_unsat  \
0                          0                            1   
1                          0                            1   
2                          0                            1   
3                          0                            0   
4                          0                            1   

   REPORTED_SATISFACTION_very_sat  REPORTED_SATISFACTION_very_unsat  \
0                               0                                 0   
1                               0                                 0   
2                               0                                 0   
3                               0                                 1   
4                               0                                 0   

   REPORTED_USAGE_LEVEL_high  REPORTED_USAGE_LEVEL_little  \
0                          0                            1   
1                          0                            0   
2                          0                            0   
3                          0                            0   
4                          0                            0   

   REPORTED_USAGE_LEVEL_very_high  REPORTED_USAGE_LEVEL_very_little  \
0                               0                                 0   
1                               0                                 1   
2                               1                                 0   
3                               1                                 0   
4                               0                                 1   

   CONSIDERING_CHANGE_OF_PLAN_considering  \
0                                       1   
1                                       0   
2                                       1   
3                                       0   
4                                       0   

   CONSIDERING_CHANGE_OF_PLAN_never_thought  CONSIDERING_CHANGE_OF_PLAN_no  \
0                                         0                              0   
1                                         0                              1   
2                                         0                              0   
3                                         1                              0   
4                                         0                              1   

   CONSIDERING_CHANGE_OF_PLAN_perhaps  
0                                   0  
1                                   0  
2                                   0  
3                                   0  
4                                   0

In [82]:
x.columns

Index(['INCOME', 'OVERAGE', 'LEFTOVER', 'HOUSE', 'HANDSET_PRICE',
       'OVER_15MINS_CALLS_PER_MONTH', 'AVERAGE_CALL_DURATION', 'COLLEGE_1',
       'REPORTED_SATISFACTION_sat', 'REPORTED_SATISFACTION_unsat',
       'REPORTED_SATISFACTION_very_sat', 'REPORTED_SATISFACTION_very_unsat',
       'REPORTED_USAGE_LEVEL_high', 'REPORTED_USAGE_LEVEL_little',
       'REPORTED_USAGE_LEVEL_very_high', 'REPORTED_USAGE_LEVEL_very_little',
       'CONSIDERING_CHANGE_OF_PLAN_considering',
       'CONSIDERING_CHANGE_OF_PLAN_never_thought',
       'CONSIDERING_CHANGE_OF_PLAN_no', 'CONSIDERING_CHANGE_OF_PLAN_perhaps'],
      dtype='object')

### 2.4 스케일링 
* 스케일링을 필요로 하는 알고리즘이 있습니다. 
* 필요하다고 판단될때 수행합니다.

In [83]:
x.describe()

INCOME       OVERAGE      LEFTOVER          HOUSE  \
count   18000.000000  18000.000000  18000.000000   18000.000000   
mean    80314.400278     86.014833     23.995500  492950.532696   
std     41703.420130     85.963981     26.860363  250664.518872   
min     20007.000000     -2.000000      0.000000  150015.000000   
25%     42151.500000      0.000000      0.000000  265148.750000   
50%     75501.500000     59.000000     14.500000  456837.500000   
75%    116082.250000    179.000000     42.000000  698473.500000   
max    159983.000000    335.000000     89.000000  999970.000000   

       HANDSET_PRICE  OVER_15MINS_CALLS_PER_MONTH  AVERAGE_CALL_DURATION  \
count   18000.000000                 18000.000000           18000.000000   
mean      390.096667                     8.012778               5.990944   
std       214.239639                     8.932820               4.398894   
min       130.000000                     0.000000               1.000000   
25%       219.000000                     1.000000               2.000000   
50%       326.000000                     4.000000               5.000000   
75%       536.000000                    15.000000              10.000000   
max       899.000000                    29.000000              15.000000   

          COLLEGE_1  REPORTED_SATISFACTION_sat  REPORTED_SATISFACTION_unsat  \
count  18000.000000               18000.000000                 18000.000000   
mean       0.502722                   0.051778                     0.199444   
std        0.500006                   0.221584                     0.399594   
min        0.000000                   0.000000                     0.000000   
25%        0.000000                   0.000000                     0.000000   
50%        1.000000                   0.000000                     0.000000   
75%        1.000000                   0.000000                     0.000000   
max        1.000000                   1.000000                     1.000000   

       REPORTED_SATISFACTION_very_sat  REPORTED_SATISFACTION_very_unsat  \
count                    18000.000000                      18000.000000   
mean                         0.253444                          0.395000   
std                          0.434995                          0.488864   
min                          0.000000                          0.000000   
25%                          0.000000                          0.000000   
50%                          0.000000                          0.000000   
75%                          1.000000                          1.000000   
max                          1.000000                          1.000000   

       REPORTED_USAGE_LEVEL_high  REPORTED_USAGE_LEVEL_little  \
count               18000.000000                 18000.000000   
mean                    0.100667                     0.393111   
std                     0.300895                     0.488455   
min                     0.000000                     0.000000   
25%                     0.000000                     0.000000   
50%                     0.000000                     0.000000   
75%                     0.000000                     1.000000   
max                     1.000000                     1.000000   

       REPORTED_USAGE_LEVEL_very_high  REPORTED_USAGE_LEVEL_very_little  \
count                    18000.000000                      18000.000000   
mean                         0.257556                          0.199667   
std                          0.437300                          0.399761   
min                          0.000000                          0.000000   
25%                          0.000000                          0.000000   
50%                          0.000000                          0.000000   
75%                          1.000000                          0.000000   
max                          1.000000                          1.000000   

       CONSIDERING_CHANGE_OF_PLAN_considering  \
count                     

In [86]:
x_numerical = x.iloc[ : , :7]
x_numerical.head(1)

INCOME  OVERAGE  LEFTOVER     HOUSE  HANDSET_PRICE  \
0   47711      183        17  730589.0            192   

   OVER_15MINS_CALLS_PER_MONTH  AVERAGE_CALL_DURATION  
0                           19                      5

In [87]:
x_list = list(x_numerical)
x_list

['INCOME',
 'OVERAGE',
 'LEFTOVER',
 'HOUSE',
 'HANDSET_PRICE',
 'OVER_15MINS_CALLS_PER_MONTH',
 'AVERAGE_CALL_DURATION']

In [88]:
from sklearn.preprocessing import MinMaxScaler

In [89]:
mm_scale = MinMaxScaler()

In [91]:
x_categorical = x.drop(x_list, axis=1)
x_categorical.head(1)

COLLEGE_1  REPORTED_SATISFACTION_sat  REPORTED_SATISFACTION_unsat  \
0          1                          0                            1   

   REPORTED_SATISFACTION_very_sat  REPORTED_SATISFACTION_very_unsat  \
0                               0                                 0   

   REPORTED_USAGE_LEVEL_high  REPORTED_USAGE_LEVEL_little  \
0                          0                            1   

   REPORTED_USAGE_LEVEL_very_high  REPORTED_USAGE_LEVEL_very_little  \
0                               0                                 0   

   CONSIDERING_CHANGE_OF_PLAN_considering  \
0                                       1   

   CONSIDERING_CHANGE_OF_PLAN_never_thought  CONSIDERING_CHANGE_OF_PLAN_no  \
0                                         0                              0   

   CONSIDERING_CHANGE_OF_PLAN_perhaps  
0                                   0

In [92]:
x_numerical = mm_scale.fit_transform(x_numerical)

In [95]:
df_numerical = pd.DataFrame(x_numerical, columns=x_list)
df_numerical

INCOME   OVERAGE  LEFTOVER     HOUSE  HANDSET_PRICE  \
0      0.197920  0.548961  0.191011  0.683064       0.080624   
1      0.386673  0.572700  0.483146  0.453056       0.284785   
2      0.931674  0.005935  0.157303  0.063520       0.717815   
3      0.997028  0.005935  0.651685  0.155248       0.655397   
4      0.024183  0.005935  0.000000  0.078465       0.133940   
...         ...       ...       ...       ...            ...   
17995  0.079099  0.599407  0.449438  0.403475       0.066320   
17996  0.837194  0.192878  0.000000  0.403475       0.854356   
17997  0.449534  0.005935  0.865169  0.403475       0.132640   
17998  0.745728  0.005935  0.213483  0.403475       0.951886   
17999  0.606118  0.667656  0.056180  0.403475       0.433030   

       OVER_15MINS_CALLS_PER_MONTH  AVERAGE_CALL_DURATION  
0                         0.655172               0.285714  
1                         0.517241               0.071429  
2                         0.000000               0.357143  
3                         0.034483               0.000000  
4                         0.000000               1.000000  
...                            ...                    ...  
17995                     0.689655               0.714286  
17996                     0.103448               0.714286  
17997                     0.000000               0.071429  
17998                     0.000000               0.071429  
17999                     0.724138               0.285714  

[18000 rows x 7 columns]

In [96]:
x_concat = pd.concat([df_numerical, x_categorical], axis=1)

In [97]:
x_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18000 entries, 0 to 17999
Data columns (total 20 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   INCOME                                    18000 non-null  float64
 1   OVERAGE                                   18000 non-null  float64
 2   LEFTOVER                                  18000 non-null  float64
 3   HOUSE                                     18000 non-null  float64
 4   HANDSET_PRICE                             18000 non-null  float64
 5   OVER_15MINS_CALLS_PER_MONTH               18000 non-null  float64
 6   AVERAGE_CALL_DURATION                     18000 non-null  float64
 7   COLLEGE_1                                 18000 non-null  uint8  
 8   REPORTED_SATISFACTION_sat                 18000 non-null  uint8  
 9   REPORTED_SATISFACTION_unsat               18000 non-null  uint8  
 10  REPORTED_SATISFACTION_very_sat    

In [98]:
x_concat.describe()

INCOME       OVERAGE      LEFTOVER         HOUSE  HANDSET_PRICE  \
count  18000.000000  18000.000000  18000.000000  18000.000000   18000.000000   
mean       0.430841      0.261172      0.269612      0.403475       0.338227   
std        0.297933      0.255086      0.301802      0.294915       0.278595   
min        0.000000      0.000000      0.000000      0.000000       0.000000   
25%        0.158202      0.005935      0.000000      0.135459       0.115735   
50%        0.396457      0.181009      0.162921      0.360987       0.254876   
75%        0.686369      0.537092      0.471910      0.645279       0.527958   
max        1.000000      1.000000      1.000000      1.000000       1.000000   

       OVER_15MINS_CALLS_PER_MONTH  AVERAGE_CALL_DURATION     COLLEGE_1  \
count                 18000.000000           18000.000000  18000.000000   
mean                      0.276303               0.356496      0.502722   
std                       0.308028               0.314207      0.500006   
min                       0.000000               0.000000      0.000000   
25%                       0.034483               0.071429      0.000000   
50%                       0.137931               0.285714      1.000000   
75%                       0.517241               0.642857      1.000000   
max                       1.000000               1.000000      1.000000   

       REPORTED_SATISFACTION_sat  REPORTED_SATISFACTION_unsat  \
count               18000.000000                 18000.000000   
mean                    0.051778                     0.199444   
std                     0.221584                     0.399594   
min                     0.000000                     0.000000   
25%                     0.000000                     0.000000   
50%                     0.000000                     0.000000   
75%                     0.000000                     0.000000   
max                     1.000000                     1.000000   

       REPORTED_SATISFACTION_very_sat  REPORTED_SATISFACTION_very_unsat  \
count                    18000.000000                      18000.000000   
mean                         0.253444                          0.395000   
std                          0.434995                          0.488864   
min                          0.000000                          0.000000   
25%                          0.000000                          0.000000   
50%                          0.000000                          0.000000   
75%                          1.000000                          1.000000   
max                          1.000000                          1.000000   

       REPORTED_USAGE_LEVEL_high  REPORTED_USAGE_LEVEL_little  \
count               18000.000000                 18000.000000   
mean                    0.100667                     0.393111   
std                     0.300895                     0.488455   
min                     0.000000                     0.000000   
25%                     0.000000                     0.000000   
50%                     0.000000                     0.000000   
75%                     0.000000                     1.000000   
max                     1.000000                     1.000000   

       REPORTED_USAGE_LEVEL_very_high  REPORTED_USAGE_LEVEL_very_little  \
count                    18000.000000                      18000.000000   
mean                         0.257556                          0.199667   
std                          0.437300                          0.399761   
min                          0.000000                          0.000000   
25%                          0.000000                          0.000000   
50%                          0.000000                          0.000000   
75%                          1.000000                          0.000000   
max                          1.000000                          1.000000   

       CONSIDERING_CHANGE_OF_PLAN_considering  \
count                            18000.0000